In [1]:
library(tidyverse)
library(repr)
library(digest)
library(gridExtra)
library(cowplot)
library(dplyr)
library(tidymodels)
library(GGally)
library(splines)

Warning message:
“package ‘tidyverse’ was built under R version 4.1.2”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.0      ✔ stringr 1.4.0 
✔ readr   2.1.3      ✔ forcats 0.5.1 
Warning message:
“package ‘ggplot2’ was built under R version 4.1.2”
Warning message:
“package ‘tibble’ was built under R version 4.1.2”
Warning message:
“package ‘tidyr’ was built under R version 4.1.2”
Warning message:
“package ‘readr’ was built under R version 4.1.2”
Warning message:
“package ‘dplyr’ was built under R version 4.1.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
“package ‘repr’ was built under R version 4.1.2”

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Warning message:
“package ‘tidym

In [2]:
setwd("/Users/matthewgillies/FantasyPreds2023")
mets <- read_csv("data/PitchMetricsBase.csv")
head(mets)

New names:
• `` -> `...28`
Rows: 1831 Columns: 28
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): last_name, first_name
dbl (25): player_id, year, p_k_percent, p_bb_percent, xslg, xwoba, xobp, xwo...
lgl  (1): ...28

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


last_name,first_name,player_id,year,p_k_percent,p_bb_percent,xslg,xwoba,xobp,xwobacon,⋯,out_zone_percent,edge_percent,whiff_percent,swing_percent,f_strike_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent,...28
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
Wainwright,Adam,425794,2021,21.0,6.0,0.388,0.302,0.305,0.353,⋯,50.5,45.4,19.7,43.8,64.4,48.7,21.5,25.4,4.4,NA
Greinke,Zack,425844,2021,17.2,5.2,0.430,0.320,0.307,0.365,⋯,54.1,42.9,21.8,46.0,66.0,45.6,23.9,25.0,5.4,NA
Santana,Ervin,429722,2021,18.8,7.9,0.533,0.369,0.339,0.427,⋯,54.5,40.0,24.7,48.0,59.8,36.8,30.8,27.4,5.0,NA
Petit,Yusmeiro,433589,2021,11.8,3.8,0.455,0.320,0.295,0.348,⋯,51.3,43.3,21.0,51.8,62.5,37.6,26.2,23.2,12.9,NA
Kintzler,Brandon,445213,2021,15.6,5.7,0.519,0.376,0.361,0.431,⋯,51.0,42.2,15.8,45.9,59.7,58.2,14.5,24.5,2.7,NA
Jansen,Kenley,445276,2021,30.9,12.9,0.270,0.260,0.295,0.310,⋯,47.6,41.5,33.1,50.1,62.8,37.9,30.1,19.0,13.1,NA


In [3]:
selected <- mets %>% select(-player_id, -"...28")
head(selected)

last_name,first_name,year,p_k_percent,p_bb_percent,xslg,xwoba,xobp,xwobacon,exit_velocity_avg,⋯,oz_swing_miss_percent,out_zone_percent,edge_percent,whiff_percent,swing_percent,f_strike_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Wainwright,Adam,2021,21.0,6.0,0.388,0.302,0.305,0.353,88.2,⋯,36.7,50.5,45.4,19.7,43.8,64.4,48.7,21.5,25.4,4.4
Greinke,Zack,2021,17.2,5.2,0.430,0.320,0.307,0.365,88.3,⋯,40.1,54.1,42.9,21.8,46.0,66.0,45.6,23.9,25.0,5.4
Santana,Ervin,2021,18.8,7.9,0.533,0.369,0.339,0.427,90.2,⋯,46.0,54.5,40.0,24.7,48.0,59.8,36.8,30.8,27.4,5.0
Petit,Yusmeiro,2021,11.8,3.8,0.455,0.320,0.295,0.348,86.5,⋯,26.8,51.3,43.3,21.0,51.8,62.5,37.6,26.2,23.2,12.9
Kintzler,Brandon,2021,15.6,5.7,0.519,0.376,0.361,0.431,91.7,⋯,31.3,51.0,42.2,15.8,45.9,59.7,58.2,14.5,24.5,2.7
Jansen,Kenley,2021,30.9,12.9,0.270,0.260,0.295,0.310,84.0,⋯,45.3,47.6,41.5,33.1,50.1,62.8,37.9,30.1,19.0,13.1


In [4]:
names <- selected %>% unite("player", last_name:first_name, sep = ", ", remove = T) %>% mutate(year = year + 1)
head(names)

player,year,p_k_percent,p_bb_percent,xslg,xwoba,xobp,xwobacon,exit_velocity_avg,launch_angle_avg,⋯,oz_swing_miss_percent,out_zone_percent,edge_percent,whiff_percent,swing_percent,f_strike_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Wainwright, Adam",2022,21.0,6.0,0.388,0.302,0.305,0.353,88.2,10.0,⋯,36.7,50.5,45.4,19.7,43.8,64.4,48.7,21.5,25.4,4.4
"Greinke, Zack",2022,17.2,5.2,0.430,0.320,0.307,0.365,88.3,10.6,⋯,40.1,54.1,42.9,21.8,46.0,66.0,45.6,23.9,25.0,5.4
"Santana, Ervin",2022,18.8,7.9,0.533,0.369,0.339,0.427,90.2,13.8,⋯,46.0,54.5,40.0,24.7,48.0,59.8,36.8,30.8,27.4,5.0
"Petit, Yusmeiro",2022,11.8,3.8,0.455,0.320,0.295,0.348,86.5,19.7,⋯,26.8,51.3,43.3,21.0,51.8,62.5,37.6,26.2,23.2,12.9
"Kintzler, Brandon",2022,15.6,5.7,0.519,0.376,0.361,0.431,91.7,3.1,⋯,31.3,51.0,42.2,15.8,45.9,59.7,58.2,14.5,24.5,2.7
"Jansen, Kenley",2022,30.9,12.9,0.270,0.260,0.295,0.310,84.0,19.8,⋯,45.3,47.6,41.5,33.1,50.1,62.8,37.9,30.1,19.0,13.1


In [5]:
fpoints <- read_csv("data/PitchFpointsper.csv")
head(fpoints)

Rows: 1836 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): player
dbl (12): year, IP, H, K, BB, ER, S, W, L, HOLD, Fpoints, Fpointsper

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


player,year,IP,H,K,BB,ER,S,W,L,HOLD,Fpoints,Fpointsper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Wainwright, Adam",2022,191.2,192,143,54,79,0,11,12,0,312.6,1.634937
"Greinke, Zack",2022,137.0,157,73,27,56,0,4,9,0,188.0,1.372263
"Verlander, Justin",2022,175.0,116,185,29,34,0,18,4,0,497.0,2.840000
"Sanchez, Anibal",2022,69.1,55,48,33,33,0,4,6,0,101.3,1.465991
"Jansen, Kenley",2022,64.0,45,85,22,24,41,5,2,0,162.0,2.531250
"Chavez, Jesse",2022,69.1,71,74,20,29,0,4,3,10,132.3,1.914616


In [6]:
fpoints_main <- fpoints %>% select("player", "year", "Fpointsper", "S")
head(fpoints_main)

player,year,Fpointsper,S
<chr>,<dbl>,<dbl>,<dbl>
"Wainwright, Adam",2022,1.634937,0
"Greinke, Zack",2022,1.372263,0
"Verlander, Justin",2022,2.840000,0
"Sanchez, Anibal",2022,1.465991,0
"Jansen, Kenley",2022,2.531250,41
"Chavez, Jesse",2022,1.914616,0


In [7]:
full_data <- right_join(names, fpoints_main, by = c("player", "year"))
head(full_data)

player,year,p_k_percent,p_bb_percent,xslg,xwoba,xobp,xwobacon,exit_velocity_avg,launch_angle_avg,⋯,edge_percent,whiff_percent,swing_percent,f_strike_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent,Fpointsper,S
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Wainwright, Adam",2022,21.0,6.0,0.388,0.302,0.305,0.353,88.2,10.0,⋯,45.4,19.7,43.8,64.4,48.7,21.5,25.4,4.4,1.634937,0
"Greinke, Zack",2022,17.2,5.2,0.430,0.320,0.307,0.365,88.3,10.6,⋯,42.9,21.8,46.0,66.0,45.6,23.9,25.0,5.4,1.372263,0
"Jansen, Kenley",2022,30.9,12.9,0.270,0.260,0.295,0.310,84.0,19.8,⋯,41.5,33.1,50.1,62.8,37.9,30.1,19.0,13.1,2.531250,41
"Chavez, Jesse",2022,27.1,8.3,0.345,0.271,0.277,0.335,90.9,10.4,⋯,41.7,17.7,43.9,65.2,43.0,29.1,26.7,1.2,1.914616,0
"Detwiler, Ross",2022,27.1,8.7,0.392,0.316,0.317,0.383,88.3,15.3,⋯,44.9,24.0,45.0,53.7,39.6,31.7,18.7,10.1,1.505747,1
"Kluber, Corey",2022,24.0,9.7,0.372,0.309,0.323,0.357,86.9,13.9,⋯,40.8,28.1,47.1,64.5,43.9,24.9,23.5,7.7,1.670732,0


In [8]:
write_csv(full_data, "data/Pitchfull.csv")